###  1. Cargue de los datos

In [1]:
import pandas as pd
import re

In [2]:
datos = pd.read_csv("C:/Users/eliza/OneDrive/Desktop/Claus/Konrad/NLP/tweets.csv")
datos.head()

,tweets,date,links
0,Uno de los puntajes nacionales es hijo de migr...,2014-12-28 00:15:41+00:00,https://twitter.com/fjnicolas/status/548995668...
1,"Es que si no hay una guerra nueva, negros famé...",2014-12-18 13:05:45+00:00,https://twitter.com/angelcalleja/status/545565...
2,"Migrantes nuestro dia, nuestra oportunidad #ta...",2014-12-18 12:39:38+00:00,https://twitter.com/Colombiaenmi/status/545559...
3,Migrantes colombianos que llegaron a Antofagas...,2014-12-17 23:30:42+00:00,https://twitter.com/ConnectasOrg/status/545360...
4,Migrantes colombianos que llegaron a Antofagas...,2014-12-17 23:30:39+00:00,https://twitter.com/ConnectasLab/status/545360...


###  2. Análisis con Flair

In [3]:
from flair.models import TextClassifier
from flair.data import Sentence

import plotly.io as pio
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

import matplotlib.pyplot as plt
%matplotlib inline
init_notebook_mode(connected=True)

from translate import Translator
from wordcloud import WordCloud
from wordcloud import ImageColorGenerator

from nltk.corpus import stopwords
stopwords = stopwords.words('spanish')

import matplotlib.pyplot as plt
import PIL.Image
from IPython.display import display

In [4]:
classifier = TextClassifier.load('sentiment')

def sent_flair(texto):
    sentence = Sentence(texto)
    classifier.predict(sentence)
    result = sentence.labels[0]
    label = result.value
    score = result.score
    if label == 'POSITIVE':
        return score
    if label == 'NEGATIVE':
        return -1 * score
    return score

datos['sent_flair'] = datos['tweets'].apply(lambda valor: sent_flair(valor))
datos.head()

2021-06-08 18:31:16,095 loading file C:\Users\eliza\.flair\models\sentiment-en-mix-distillbert_4.pt


,tweets,date,links,sent_flair
0,Uno de los puntajes nacionales es hijo de migr...,2014-12-28 00:15:41+00:00,https://twitter.com/fjnicolas/status/548995668...,0.997409
1,"Es que si no hay una guerra nueva, negros famé...",2014-12-18 13:05:45+00:00,https://twitter.com/angelcalleja/status/545565...,0.966135
2,"Migrantes nuestro dia, nuestra oportunidad #ta...",2014-12-18 12:39:38+00:00,https://twitter.com/Colombiaenmi/status/545559...,0.839582
3,Migrantes colombianos que llegaron a Antofagas...,2014-12-17 23:30:42+00:00,https://twitter.com/ConnectasOrg/status/545360...,-0.655836
4,Migrantes colombianos que llegaron a Antofagas...,2014-12-17 23:30:39+00:00,https://twitter.com/ConnectasLab/status/545360...,-0.796346


In [5]:
print("Tweet más negativo")
datos.sort_values('sent_flair').iloc[0]['tweets']

Tweet más negativo


'Por qué el periodista de @T13 le pide documentación a migrantes colombianos? No le corresponde exponerlos/as así!'

In [6]:
print("Tweet más positivo")
datos.sort_values('sent_flair').iloc[-1]['tweets']

Tweet más positivo


'@Guarenas27 @EcuanimidadM @aIyllucci REVISIÓN URGENTE D LA SITUACIÓN D MIGRANTES COLOMBIANOS AUNQUE LOS DOMINIQUIS  TIENEN LO SUYO ¡OJO!'

In [7]:
#Cantidad de tweets positivos y negativos
datos['sent_flair_label'] = datos['sent_flair'].apply(lambda valor: 'POSITIVO' if valor >=0 else 'NEGATIVO')
datos['sent_flair_label'].value_counts()

POSITIVO    112
NEGATIVO     89
Name: sent_flair_label, dtype: int64

In [8]:
#Diagramación
temp = pd.DataFrame(datos['sent_flair_label'].value_counts()).reset_index()
temp.columns = ['sentimiento', 'cuántos']

trace = go.Bar(x=temp.sentimiento.values,
               y=temp.cuántos.values,
               marker=dict(color=["blue","orange"]))

layout = go.Layout(title='Sentimientos')

fig = go.Figure(data=[trace], layout=layout)
iplot(fig)

### 3. Análisis con Vader

In [9]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from textblob  import TextBlob

In [10]:
tweets_blob = [TextBlob(tweet) for tweet in datos['tweets']]
datos['Polaridad'] = [b.sentiment.polarity for b in tweets_blob]
datos['Subjetividad'] = [b.sentiment.subjectivity for b in tweets_blob]

In [11]:
analyzer = SentimentIntensityAnalyzer()
analyzer.polarity_scores(datos.tweets)

{'neg': 0.012, 'neu': 0.986, 'pos': 0.002, 'compound': -0.9728}

In [12]:
analyzer = SentimentIntensityAnalyzer()
 
datos['Compound'] = [analyzer.polarity_scores(v)['compound'] for v in datos['tweets']]
datos['Neg'] = [analyzer.polarity_scores(v)['neg'] for v in datos['tweets']]
datos['Neu'] = [analyzer.polarity_scores(v)['neu'] for v in datos['tweets']]
datos['Pos'] = [analyzer.polarity_scores(v)['pos'] for v in datos['tweets']]
datos

,tweets,date,links,sent_flair,sent_flair_label,Polaridad,Subjetividad,Compound,Neg,Neu,Pos
0,Uno de los puntajes nacionales es hijo de migr...,2014-12-28 00:15:41+00:00,https://twitter.com/fjnicolas/status/548995668...,0.997409,POSITIVO,0.000000,0.000000,0.0000,0.000,1.000,0.0
1,"Es que si no hay una guerra nueva, negros famé...",2014-12-18 13:05:45+00:00,https://twitter.com/angelcalleja/status/545565...,0.966135,POSITIVO,0.000000,0.000000,-0.5267,0.161,0.839,0.0
2,"Migrantes nuestro dia, nuestra oportunidad #ta...",2014-12-18 12:39:38+00:00,https://twitter.com/Colombiaenmi/status/545559...,0.839582,POSITIVO,0.000000,0.000000,0.0000,0.000,1.000,0.0
3,Migrantes colombianos que llegaron a Antofagas...,2014-12-17 23:30:42+00:00,https://twitter.com/ConnectasOrg/status/545360...,-0.655836,NEGATIVO,0.000000,0.000000,0.0000,0.000,1.000,0.0
4,Migrantes colombianos que llegaron a Antofagas...,2014-12-17 23:30:39+00:00,https://twitter.com/ConnectasLab/status/545360...,-0.796346,NEGATIVO,0.000000,0.000000,0.0000,0.000,1.000,0.0
...,...,...,...,...,...,...,...,...,...,...,...
196,Los colombianos migrantes necesitan un preside...,2014-04-26 19:31:16+00:00,https://twitter.com/SantiagoPeye/status/460139...,-0.552418,NEGATIVO,0.000000,0.000000,0.0000,0.000,1.000,0.0
197,@ameliarueda hay prioridades mas grandes que ...,2014-04-25 03:50:51+00:00,https://twitter.com/erickvillalobos/status/459...,0.819401,POSITIVO,0.000000,0.000000,0.0000,0.000,1.000,0.0
198,MINTRABAJO IMPLEMENTARÁ MEDIDAS DE PROTECCIÓN ...,2014-04-22 21:59:34+00:00,https://twitter.com/spacheco68/status/45872683...,0.688444,POSITIVO,0.000000,0.000000,0.0000,0.000,1.000,0.0
199,Diseñamos esta GUIA PARA MIGRANTES COLOMBIANOS...,2014-04-15 16:18:40+00:00,https://twitter.com/afro_candombera/status/456...,-0.994761,NEGATIVO,0.000000,0.000000,0.0000,0.000,1.000,0.0
